# Support

If you have any questions or want to report an issue regarding the Python SDK please email <a href="mailto:clarion-support@coremont.com?subject=Clarion API/SDK Support Query&body=Your Email Account for the API (if not your own):%0D%0ADate/time of the issue:%0D%0ARelevant section of the log file:%0D%0ASDK Version:%0D%0AWhat you expected to happen and what actually happened:%0D%0AAny other relevant information:%0D%0A">Clarion Support</a> with the following information:

- Your email account for the API (in the case that it is not the same as the originating email address),
- Date/time of the issue,
- The relevant section of the log file (see [Logging](https://cm.coremont.com/docs/api/sdk/details#logging) section) at the time the issue occurred, which will contain the [version](https://cm.coremont.com/docs/api/sdk/details#versioning) and correlation ID,
- What you expected to happen and what actually happened,
- Any other relevant information.

# Typical workflow for scenario analysis
1. Create one or more shifts using the create_shift function. The list of available shifts with required parameters can be found
    [here](https://cm.coremont.com/docs/api/sdk/functions#create_shift).
2. Combine the shifts into a scenario object.
3. Identify the set of risk metrics of interest. The metrics function can be used to get a list of all supported metrics with their config.
4. Run the risk function to invoke the scenarios on the chosen portfolio and date (can be live or historic), to return the selected risk metrics.

In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from dotenv import load_dotenv
load_dotenv();

In [9]:
import base64
import json
import logging
import os
import re
import sys
import time
import threading
from functools import partial
from itertools import chain
from pathlib import Path
from uuid import uuid4

import bhutils
import blpapi
import ipyvuetify as v
import ipywidgets as w
import numpy as np
import pandas as pd
import clarion
import QuantLib as ql
import dataframe_image as dfi
import win32com.client as win32
from bqplot import ColorScale, LinearScale, OrdinalColorScale, OrdinalScale
from ipydatagrid import BarRenderer, DataGrid, Expr, TextRenderer, VegaExpr
from IPython.display import display, clear_output
from pandas.tseries.offsets import BDay
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from blp.blp import BlpParser


from gioutils.clarion_ext import Runner
from gioutils.gui.pandas import millifyp as millify
from gioutils import ezutils as ez
from gioutils.blpw import BlpParser, BlpQuery
from gioutils.clarion_ext import super_clarion
from gioutils.tools import parse_swap_periods
from gioutils.ezutils import (
    bh,
    bhs,
    get_id,
    get_price_info,
)

from gioutils.utils import (
    get_bday,
    get_next_n,
    parse_offset,
    today,
    get_generic_future, 
    get_bbg_fut_chain_ticker,
    get_root
)

pd.options.plotting.backend = "plotly"
pd.options.display.float_format = partial(millify, pct=None, decimal=4)

In [10]:
from gioutils.tools_clarion import get_clarion_positions

In [11]:
clarion.server_login()
bq = BlpQuery(timeout=50000, parser=BlpParser(raise_security_errors=False)).start()
# ba = BHSysApi(password="A&3jg5Aj")

2023-02-27 11:46:50,497 - Authenticating...
2023-02-27 11:46:51,001 - Successfully authenticated


In [12]:
from IPython.display import display, HTML

## Create one or more shifts and combine it into a scenario object

Documentation on supported shifts is [here](https://cm.coremont.com/docs/api/sdk/functions#create_shift).

Documentation on scenarios is [here](https://cm.coremont.com/docs/api/sdk/functions#create_scenario).
With FX, you can also combine a spot and vol shift in a single scenario.

In [13]:
usd_curve_shift = clarion.create_shift('RatesCurveShift', 'Relative', 0.01, {'curveFamily': 'USD'})

FxSpotShift = clarion.create_shift('FxSpotShift','Relative', 0.05, {'currency': 'EUR'})
FxVolatilityShift = clarion.create_shift('FxVolatilityShift', 'parallel', -0.025, {'currencypair': 'EURUSD'})

scenario_rates = clarion.create_scenario('ScenRates', [usd_curve_shift])
scenario_fx = clarion.create_scenario('ScenFx', [FxSpotShift, FxVolatilityShift])

## Identify metrics of interest

In [14]:
# Optional. 
import pandas as pd
pd.set_option('display.max_colwidth', None)   #<-- this gets rid of any width size restrictions
pd.set_option('display.max_rows', None)  #<-- this gets rid of any restrictions on the num of rows displayed
pd.set_option('display.max_columns', None)  #<-- this gets rid of any restrictions on the num of columns displayed

In [15]:
# See what metrics and options for config are available
clarion.risk_metrics()

metricType assetClass  \
0      IntrinsicValue        Any   
1   LocalPresentValue        Any   
2         OptionTheta        Any   
3         OptionTheta        Any   
4         OptionTheta        Any   
5        PresentValue        Any   
6               Theta        Any   
7           TimeValue        Any   
8     CorrelationRisk     Credit   
9                Cs01     Credit   
10              Delta     Credit   
11              Gamma     Credit   
12               Vega     Credit   
13              Delta     Equity   
14       DeltaPercent     Equity   
15      DividendDelta     Equity   
16              Gamma     Equity   
17                Rho     Equity   
18               Spot     Equity   
19              Vanna     Equity   
20               Vega     Equity   
21              Volga     Equity   
22              Carry         Fx   
23              Carry         Fx   
24              Delta         Fx   
25              Delta         Fx   
26              Gamma         Fx   
27              Gamma         Fx   
28               Spot         Fx   
29               Vega         Fx   
30               Vega         Fx   
31               Vega         Fx   
32         Volatility         Fx   
33         Volatility         Fx   
34              Delta      Rates   
35              Gamma      Rates   
36     InflationDelta      Rates   
37               Vega      Rates   

                                                                             config  
0                                                                                    
1                                                                                    
2                                                                   {'Period': '1'}  
3                                                                   {'Period': '7'}  
4                                                                  {'Period': '30'}  
5                                                                                    
6                                                                                    
7                                                                                    
8                                                                                    
9                                                                                    
10                                                                                   
11                                                                                   
12                                                                                   
13                                               {'VolSpotDynamic': 'StickyStrike'}  
14                                               {'VolSpotDynamic': 'StickyStrike'}  
15                                                                                   
16                                               {'VolSpotDynamic': 'StickyStrike'}  
17                                               {'VolSpotDynamic': 'StickyStrike'}  
18                                                                                   
19                                               {'VolSpotDynamic': 'StickyStrike'}  
20                                                                                   
21                                                                                   
22                                            {'Period': '1', 'StartBumpDays': '0'}  
23                                            {'Period': '7', 'StartBumpDays': '0'}  
24                                                {'VolSpotDynamic': 'StickyDelta'}  
25                                               {'VolSpotDynamic': 'StickyStrike'}  
26                                                {'VolSpotDynamic': 'StickyDelta'}  
27                                               {'VolSpotDynamic': 'StickyStrike'}  
28                                                                                   
29                        

In [16]:
# Create a list of metrics in form clarion.metric('label', 'metricType', 'assetClass', {config})
m1 = clarion.metric('PV', 'PresentValue', 'Any')
m2 = clarion.metric('FxDelta', 'Delta', 'Fx', {'VolSpotDynamic': 'StickyDelta'})
m3 = clarion.metric('RatesDelta', 'Delta', 'Rates', {'VolSpotDynamic': 'StickyDelta', 'RiskSpace': 'Par', 'RiskBuckets': 'Default'})
fx_riskmetrics = [m1,m2]
rates_riskmetrics = [m1,m3]

## Filter positions of interest from a book using the asset class filter

In [17]:
rates = clarion.create_trade('Risk_Trade', asset_class='Rates', portfolio='MM-BAL')
x1_tst_fx_trades = clarion.create_trade('Risk_Trade', asset_class='FX', portfolio='X1-TST')

In [23]:
pv = clarion.metric('pv', 'PresentValue', 'Any')

In [20]:
book = clarion.create_trade('mm',  portfolio='MM-BAL')

In [24]:
res = clarion.price.run([m3, pv], [rates], start_date='2018-01-01', end_date='2023-02-24', market_mode='live')

2023-02-24 11:42:00,492 - Response Count Summary - DFRAME:0, ERROR:1


{'DFRAME': None,
 'IS_ERROR': None,
 'HAS_ERROR': None,
 'JSON': None,
 'TIMEOUT': None,
 'ERROR': 'HTTPError occured 400 Client Error: Bad Request for url: https://api.clarion.coremont.com/v2/price. HTTPErrorCode 400. {"type":"https://tools.ietf.org/html/rfc7231#section-6.5.1","title":"One or more validation errors occurred.","status":400,"traceId":"00-0a5ae6bba298fedbddfe0c9bde078ec6-fb7da5a2e97ddeab-01","errors":{"TradeDefinitions[0].Trade.Instrument.InstrumentType":["The InstrumentType field is required."]}}',
 'RESPONSE_CATEGORY': None,
 'KEY': None}

## Invoke the risks.run function with positions and scenario

In [22]:
#Run the fx scenario on the fx positions
response = clarion.risks.run(
    rates_riskmetrics, 
    rates, 
    # [scenario_fx], 
    # asof_date='2022-09-01', 
    market_mode='live'
)
if response['ERROR']:
    print(response['ERROR'])
else:
    display(response['DFRAME'])

2023-02-27 22:32:22,650 - Requesting data
2023-02-27 22:32:39,927 - Request complete
2023-02-27 22:32:39,940 - Response Count Summary - DFRAME:110


scenario_id         id  status  isIntraday                   type  \
0    Rates positions  275800481    Live       False           CmIdentifier   
1    Rates positions  275800531    Live       False           CmIdentifier   
2    Rates positions  275806141    Live       False           CmIdentifier   
3    Rates positions  281310021    Live       False           CmIdentifier   
4    Rates positions  281310552    Live       False           CmIdentifier   
5    Rates positions  282039341    Dead       False           CmIdentifier   
6    Rates positions  282039341  Offset       False           CmIdentifier   
7    Rates positions  282182932    Live       False           CmIdentifier   
8    Rates positions  282182941    Live       False           CmIdentifier   
9    Rates positions  282600241    Dead       False           CmIdentifier   
10   Rates positions  282600241  Offset       False           CmIdentifier   
11   Rates positions  282889582    Live       False           CmIdentifier   
12   Rates positions  282889591    Live       False           CmIdentifier   
13   Rates positions  283194131    Live       False           CmIdentifier   
14   Rates positions  283194142    Live       False           CmIdentifier   
15   Rates positions  283194151    Live       False           CmIdentifier   
16   Rates positions  283194192    Live       False           CmIdentifier   
17   Rates positions  283262441    Live       False           CmIdentifier   
18   Rates positions  283883092    Live       False           CmIdentifier   
19   Rates positions  283883102    Live       False           CmIdentifier   
20   Rates positions  283883111    Live       False           CmIdentifier   
21   Rates positions  283883141    Live       False           CmIdentifier   
22   Rates positions  284252581    Live       False           CmIdentifier   
23   Rates positions  284252602    Live       False           CmIdentifier   
24   Rates positions  284556951    Live       False           CmIdentifier   
25   Rates positions  284556962    Live       False           CmIdentifier   
26   Rates positions  284556971    Live       False           CmIdentifier   
27   Rates positions  284737802    Live       False           CmIdentifier   
28   Rates positions  284737811    Live       False           CmIdentifier   
29   Rates positions  284811311    Live       False           CmIdentifier   
30   Rates positions  284815222    Live       False           CmIdentifier   
31   Rates positions  284876171    Live       False           CmIdentifier   
32   Rates positions  284876192    Live       False           CmIdentifier   
33   Rates positions  285073911    Dead       False           CmIdentifier   
34   Rates positions  285073911  Offset       False           CmIdentifier   
35   Rates positions  285379211    Live       False           CmIdentifier   
36   Rates positions  285379222    Live       False           CmIdentifier   
37   Rates positions  285404172    Live       False           CmIdentifier   
38   Rates positions  285404191    Live       False           CmIdentifier   
39   Rates positions  285854102    Live       False           CmIdentifier   
40   Rates positions  285860261    Live       False           CmIdentifier   
41   Rates positions  285861392    Live       False           CmIdentifier   
42   Rates positions  285862001    Live       False           CmIdentifier   
43   Rates positions  285862012    Live       False           CmIdentifier   
44   Rates positions  285862021    Live       False           CmIdentifier   
45   Rates positions  285862031    Live       False           CmIdentifier   
46   Rates positions  285862042    Live       False           CmIdentifier   
47   Rates positions  285862052    Live       False           CmIdentifier   
48   Rates positions  285871091    Live       False           CmIdentifier   
49   Rates positions  285871101    Live       False           CmIdentifier   
50   Rates positions  285871142

In [20]:
i = 3
step = 10

In [26]:
response['DFRAME']

scenario_id         id   status  isIntraday                   type  \
0    Rates positions  275800481     Live       False           CmIdentifier   
1    Rates positions  275800531     Live       False           CmIdentifier   
2    Rates positions  275806141     Live       False           CmIdentifier   
3    Rates positions  277848461     Live       False           CmIdentifier   
4    Rates positions  277848631     Live       False           CmIdentifier   
5    Rates positions  279309851     Live       False           CmIdentifier   
6    Rates positions  279309861     Live       False           CmIdentifier   
7    Rates positions  281310021     Live       False           CmIdentifier   
8    Rates positions  281310552     Live       False           CmIdentifier   
9    Rates positions  282039331     Live       False           CmIdentifier   
10   Rates positions  282039341     Live       False           CmIdentifier   
11   Rates positions  282182932     Live       False           CmIdentifier   
12   Rates positions  282182941     Live       False           CmIdentifier   
13   Rates positions  282286381     Live       False           CmIdentifier   
14   Rates positions  282286391     Live       False           CmIdentifier   
15   Rates positions  282600241     Live       False           CmIdentifier   
16   Rates positions  282600261     Live       False           CmIdentifier   
17   Rates positions  282889582     Live       False           CmIdentifier   
18   Rates positions  282889591     Live       False           CmIdentifier   
19   Rates positions  283137551     Live       False           CmIdentifier   
20   Rates positions  283137571     Live       False           CmIdentifier   
21   Rates positions  283194131     Live       False           CmIdentifier   
22   Rates positions  283194142     Live       False           CmIdentifier   
23   Rates positions  283194151     Live       False           CmIdentifier   
24   Rates positions  283194192     Live       False           CmIdentifier   
25   Rates positions  283262441     Live       False           CmIdentifier   
26   Rates positions  283883092     Live       False           CmIdentifier   
27   Rates positions  283883102     Live       False           CmIdentifier   
28   Rates positions  283883111     Live       False           CmIdentifier   
29   Rates positions  283883141     Live       False           CmIdentifier   
30   Rates positions  284252581     Live       False           CmIdentifier   
31   Rates positions  284252602     Live       False           CmIdentifier   
32   Rates positions  284556951     Live       False           CmIdentifier   
33   Rates positions  284556962     Live       False           CmIdentifier   
34   Rates positions  284556971     Live       False           CmIdentifier   
35   Rates positions  284737802     Live       False           CmIdentifier   
36   Rates positions  284737811     Live       False           CmIdentifier   
37   Rates positions  284811311     Live       False           CmIdentifier   
38   Rates positions  284815222     Live       False           CmIdentifier   
39   Rates positions  284876171     Live       False           CmIdentifier   
40   Rates positions  284876192     Live       False           CmIdentifier   
41   Rates positions  285073821     Live       False           CmIdentifier   
42   Rates positions  285073861     Live       False           CmIdentifier   
43   Rates positions  285073881     Live       False           CmIdentifier   
44   Rates positions  285073901     Live       False           CmIdentifier   
45   Rates positions  285073911     Live       False           CmIdentifier   
46   Rates positions  285073921     Live       False           CmIdentifier   
47   Rates positions  285379211     Live       False           CmIdentifier   
48   Rates positions  285379222     Live       False           CmIdentifier   
49   Rates positions  285404172     Live       False       

In [ ]:
from datetime import date
today = date.today().strftime('%Y-%m-%d')  

In [ ]:
#Run the rates scenario on the rates positions as of live market data and positions
response = clarion.risks.run(rates_riskmetrics, x1_tst_rates_trades, [scenario_rates], asof_date=today, market_mode='live')
if response['ERROR']:
    print(response['ERROR'])
else:
    display(response['DFRAME'])

## Example of the retrieve function
#### Can be used in the case where a request has timed out and requires a re-run or any situation in which you have a key for data and wish to retrieve that datan

In [ ]:
response_result = clarion.risks.retrieve(response['KEY'])